# EXTRA EXERCISE 3

By using the dataset `general_motors.csv`, design an I-MR control chart with probability limits (i.e., use the true distribution of both statistics) with $\alpha=0.01$.

With regard to the MR chart, use both the half-normal distribution and the data transformation.



In [ ]:
# Import the necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
import qdatoolkit as qda

# Import the dataset
data = pd.read_csv('../../Data/general_motors.csv')

# Inspect the dataset
data.head()

> Let's check normality and independence.

In [ ]:
qda.Assumptions(data['GM']).normality()

In [ ]:
_ = qda.Assumptions(data['GM']).independence()

> Let's find the new value of K for designing the IMR control chart. 

In [ ]:
alpha = 0.01
K_alpha = stats.norm.ppf(1-alpha/2)
print('New K value = %.3f' % K_alpha)

> Remove the OOC we observed in EXTRA EXERCISE 2.

In [ ]:
OOC_idx = 28
data['GM'].iloc[OOC_idx] = np.nan

In [ ]:
# Design the IMR control chart
data_IMR = qda.ControlCharts.IMR(data, 'GM', K = K_alpha)

> Let's redesign the MR control chart by using the half-normal approximation first:
> - $UCL = D_{1-\alpha/2} \frac{\bar{MR}}{d_2}$
> - $LCL = D_{\alpha/2} \frac{\bar{MR}}{d_2}$
>
> For $n=2$ (Alwan, Appendix A):
> - $D_{1-\alpha/2} = \sqrt{2} z_{alpha/4}$
> - $D_{\alpha/2} = \sqrt{2} z_{1/2 - alpha/4}$

In [ ]:
D_UCL = np.sqrt(2) * stats.norm.ppf(1-alpha/4)
D_LCL = np.sqrt(2) * stats.norm.ppf(1 - (1/2 - alpha/4))

MR_UCL = D_UCL * data_IMR['MR'].mean()/qda.constants.getd2(2)
MR_LCL = D_LCL * data_IMR['MR'].mean()/qda.constants.getd2(2)

print('MR_UCL = %.4f' % MR_UCL)
print('MR_LCL = %.4f' % MR_LCL)

> Let's plot the MR chart with the half-normal approximation.

In [ ]:
data_IMR['MR_TEST1'] = np.where((data_IMR['MR'] > MR_UCL) | 
                                (data_IMR['MR'] < MR_LCL), data_IMR['MR'], np.nan)

plt.plot(data_IMR['MR'], 'o-')
plt.axhline(MR_UCL, color = 'r')
plt.axhline(MR_LCL, color = 'r')
plt.plot(data_IMR['MR_TEST1'], linestyle='none', marker='s', color='r', markersize=10)
plt.show()